In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from betedge_data import BetEdgeClient
from betedge_data import ClientOptionRequest

client = BetEdgeClient()

request = ClientOptionRequest(
    root="AAPL",
    start_date="20240101",
    end_date="20240131",
    data_schema="quote",
    interval=900000,  # 15 minutes in milliseconds
    force_refresh=False,
)

job_info = client.request_data(request)

print(job_info)

JobInfo(job_id=UUID('0206947f-af31-4f2c-9352-02d340e15578'), status=<JobStatus.COMPLETED: 'completed'>, total_items=21, completed_items=21, error_message=None, created_at=datetime.datetime(2025, 9, 26, 20, 15, 47, 954434, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 9, 26, 20, 15, 47, 954434, tzinfo=datetime.timezone.utc))


In [27]:
df = client.retrieve_data(request)

In [37]:
import polars as pl
df.filter((pl.col("bid_size") > 0) & (pl.col("ask_size") > 0)).count()

ms_of_day,bid_size,bid_exchange,bid,bid_condition,ask_size,ask_exchange,ask,ask_condition,date,root,expiration,strike,right
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937


In [ ]:
df = df.filter(
    (pl.col("bid_size") > 0) & (pl.col("ask_size") > 0)
                ).with_columns([
                    ((pl.col("bid") + pl.col("ask"))/2).alias("mid"),
                    (pl.col("bid") - pl.col("ask")).abs().alias("spread"),
                ])


ms_of_day,bid_size,bid_exchange,bid,bid_condition,ask_size,ask_exchange,ask,ask_condition,date,root,expiration,strike,right,mid,spread
u32,u32,u32,f32,u32,u32,u32,f32,u32,u32,str,u32,u32,str,f32,f32
35100000,34,31,134.649994,50,14,6,138.100006,50,20240102,"""AAPL""",20240119,50000,"""C""",136.375,3.450012
36000000,30,60,134.800003,50,30,60,139.0,50,20240102,"""AAPL""",20240119,50000,"""C""",136.899994,4.199997
36900000,30,73,136.5,50,30,73,136.899994,50,20240102,"""AAPL""",20240119,50000,"""C""",136.699997,0.399994
37800000,33,43,136.100006,50,34,47,136.550003,50,20240102,"""AAPL""",20240119,50000,"""C""",136.325012,0.449997
38700000,10,47,135.5,50,30,47,135.800003,50,20240102,"""AAPL""",20240119,50000,"""C""",135.649994,0.300003
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
54000000,3,1,185.970001,0,3,60,185.990005,0,20240131,"""AAPL""",0,0,"""U""",185.980011,0.020004
54900000,1,60,185.490005,0,1,1,185.5,0,20240131,"""AAPL""",0,0,"""U""",185.494995,0.009995
55800000,8,65,185.509995,0,3,1,185.529999,0,20240131,"""AAPL""",0,0,"""U""",185.519989,0.020004


In [42]:
stock = df.filter(pl.col("expiration") == 0)
joined = df.filter(pl.col("expiration") != 0).join(
    stock, on=["ms_of_day", "date"]
)
joined

ms_of_day,bid_size,bid_exchange,bid,bid_condition,ask_size,ask_exchange,ask,ask_condition,date,root,expiration,strike,right,mid,spread,bid_size_right,bid_exchange_right,bid_right,bid_condition_right,ask_size_right,ask_exchange_right,ask_right,ask_condition_right,root_right,expiration_right,strike_right,right_right,mid_right,spread_right
u32,u32,u32,f32,u32,u32,u32,f32,u32,u32,str,u32,u32,str,f32,f32,u32,u32,f32,u32,u32,u32,f32,u32,str,u32,u32,str,f32,f32
35100000,34,31,134.649994,50,14,6,138.100006,50,20240102,"""AAPL""",20240119,50000,"""C""",136.375,3.450012,1,1,186.25,0,7,60,186.259995,0,"""AAPL""",0,0,"""U""",186.255005,0.009995
36000000,30,60,134.800003,50,30,60,139.0,50,20240102,"""AAPL""",20240119,50000,"""C""",136.899994,4.199997,3,65,186.509995,0,3,60,186.529999,0,"""AAPL""",0,0,"""U""",186.519989,0.020004
36900000,30,73,136.5,50,30,73,136.899994,50,20240102,"""AAPL""",20240119,50000,"""C""",136.699997,0.399994,4,3,186.520004,0,1,1,186.529999,0,"""AAPL""",0,0,"""U""",186.524994,0.009995
37800000,33,43,136.100006,50,34,47,136.550003,50,20240102,"""AAPL""",20240119,50000,"""C""",136.325012,0.449997,2,7,186.179993,0,4,3,186.190002,0,"""AAPL""",0,0,"""U""",186.184998,0.01001
38700000,10,47,135.5,50,30,47,135.800003,50,20240102,"""AAPL""",20240119,50000,"""C""",135.649994,0.300003,4,68,185.520004,0,6,3,185.539993,0,"""AAPL""",0,0,"""U""",185.529999,0.019989
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
54000000,51,7,191.550003,50,50,1,195.75,50,20240131,"""AAPL""",20250321,380000,"""P""",193.649994,4.199997,3,1,185.970001,0,3,60,185.990005,0,"""AAPL""",0,0,"""U""",185.980011,0.020004
54900000,56,1,192.149994,50,50,1,196.800003,50,20240131,"""AAPL""",20250321,380000,"""P""",194.475006,4.650009,1,60,185.490005,0,1,1,185.5,0,"""AAPL""",0,0,"""U""",185.494995,0.009995
55800000,50,1,193.0,50,50,60,196.850006,50,20240131,"""AAPL""",20250321,380000,"""P""",194.925003,3.850006,8,65,185.509995,0,3,1,185.529999,0,"""AAPL""",0,0,"""U""",185.519989,0.020004


In [ ]:
import httpx

subreqs = request.get_subrequests()

r = subreqs[0]
url = r.get_url()
http_client = httpx.Client(
    timeout=60,
    limits=httpx.Limits(
        max_connections=4,
        max_keepalive_connections=4,
    ),
    transport=httpx.HTTPTransport(retries=0),
    http2=True,
    headers={},
)

response = http_client.get(url)
print(url)
print(response)

http://127.0.0.1:25510/v2/bulk_hist/option/quote?root=AAPL&exp=0&start_date=20240102&end_date=20240102&ivl=900000
<Response [200 OK]>
